# Quoridor Game Rules (Implementation)
## Board

9 × 9 grid of cells.

Rows numbered 1–9 from top to bottom.

Columns numbered 1–9 from left to right.

## Players

Two players: MAX (player 1) and MIN (player 2).

Each player has:

- One pawn.

- 10 walls.

### Starting Positions

MAX pawn: row 1, column 5.

MIN pawn: row 9, column 5.

### Objectives

MAX wins by reaching row 9.

MIN wins by reaching row 1.

## Pawns

Can move to adjacent squares: up, down, left, right.

Can jump over opponent if directly adjacent and no wall in between.

Can move diagonally if the direct jump over opponent is blocked by a wall.

Cannot move outside the board or into a square blocked by a wall.

## Walls

Each player can place walls until they have none left.

Walls can be horizontal (H) or vertical (V).

Each wall blocks two passages between cells.

Walls cannot overlap any existing wall.

Walls cannot completely block a path for either player; there must always be at least one valid path from each pawn to its goal row.

Once placed, walls cannot be moved or removed.

## Turns

Players alternate turns.

On a turn, a player may:

Move their pawn according to the movement rules.

Place a wall according to the placement rules.

## Game End

The game ends when one pawn reaches the opponent’s starting row.

The player who reaches the goal row first wins.

## Notes

Anchors (row, column) are an implementation detail for walls, representing the top-left cell of a wall.

Legal moves are automatically calculated to enforce rules.

Wall placement conflicts and path-blocking are checked programmatically.

# AI Game Agent Overview

**State and Action Designer**  
Defines the game environment, including pawns, walls, and legal moves. It utilizes a frozen dataclass for the state to enable efficient hashing and state-space exploration, ensuring structured transitions between players.

**Heuristic Evaluation**  
Computes the game-state value based on the relative distance to the goal. It employs an optimized Breadth-First Search (BFS) that pre-calculates edge connectivity based on current wall positions, significantly reducing the overhead of pathfinding during deep searches. We minimize the usage of walls and also prefer going forward instead of backward.

**Minimax Implementation**  
Recursive search algorithm that explores all possible moves to a specified depth, selecting the optimal move assuming both players act rationally. Counts examined nodes for performance tracking.

**Alpha−Beta Pruning**  
An optimized recursive search that identifies the best move by maintaining $\alpha$ (the best score for the maximizer) and $\beta$ (the best score for the minimizer). By passing these bounds through the search tree, the agent eliminates subtrees that cannot influence the final decision, drastically improving performance.

**Move Ordering & Performance**
To maximize pruning efficiency, the agent uses a priority-based move sorter at each node. By exploring promising moves first—such as pawn movements toward the goal—the $\alpha$ and $\beta$ bounds tighten earlier in the search. The system tracks examined_nodes and pruned_nodes to quantify the computational savings gained from these optimizations.


### Imports

In [ ]:
from state import State
from logic import legal_moves, apply, is_terminal
import random
from playpiece import Pawn, Wall
  
from collections import deque

In [25]:
state = State.get_initial_state(random_turn=True)
print(state)
state.display()

State(p1=Pawn(row=1, col=5, owner='MAX'), p2=Pawn(row=9, col=5, owner='MIN'), p1_walls=10, p2_walls=10, walls=frozenset(), active_player='MAX')
   1 2 3 4 5 6 7 8 9
1 .   .   .   .   1   .   .   .   .
                                   
2 .   .   .   .   .   .   .   .   .
                                   
3 .   .   .   .   .   .   .   .   .
                                   
4 .   .   .   .   .   .   .   .   .
                                   
5 .   .   .   .   .   .   .   .   .
                                   
6 .   .   .   .   .   .   .   .   .
                                   
7 .   .   .   .   .   .   .   .   .
                                   
8 .   .   .   .   .   .   .   .   .
                                   
9 .   .   .   .   2   .   .   .   .


## Heurisitic

In [44]:
def evaluation(state: State) -> float:
    def get_blocked_edges(state: State):
        blocked = set()
        # Only check neighbors to keep the set small
        for r in range(1, 10):
            for c in range(1, 10):
                for dr, dc in [(1, 0), (0, 1)]: # Only check South and East to avoid duplicates
                    nr, nc = r + dr, c + dc
                    if 1 <= nr <= 9 and 1 <= nc <= 9:
                        for w in state.walls:
                            if w.blocks_edge((r, c), (nr, nc)):
                                blocked.add(((r, c), (nr, nc)))
                                blocked.add(((nr, nc), (r, c)))
        return blocked

    blocked_edges = get_blocked_edges(state)

    def shortest_path(pawn: Pawn, goal_row: int) -> int:
        q = deque([(pawn.row, pawn.col, 0)])
        seen = {(pawn.row, pawn.col)}
        while q:
            r, c, d = q.popleft()
            if r == goal_row: return d
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = r + dr, c + dc
                if 1 <= nr <= 9 and 1 <= nc <= 9:
                    if (nr, nc) not in seen and ((r, c), (nr, nc)) not in blocked_edges:
                        seen.add((nr, nc))
                        q.append((nr, nc, d + 1))
        return 100

    max_dist = shortest_path(state.p1, 9)
    min_dist = shortest_path(state.p2, 1)
    
    score = min_dist - max_dist
    score += (state.p1_walls * 0.8)
    score -= (state.p2_walls * 0.8)
    score += (state.p1.row * 0.1)
    score -= ((10 - state.p2.row) * 0.1)

    return score

### MINIMAX Implementation

In [37]:
examined_nodes = 0

def minimax(state: "State", depth: int, maximizing: bool) -> float:
    global examined_nodes
    examined_nodes += 1

    if depth == 0 or is_terminal(state):
        return evaluation(state)

    moves = legal_moves(state)

    if maximizing:
        max_eval = -float('inf')
        for move in moves:
            new_state = apply(state, move)
            val = minimax(new_state, depth-1, False)
            max_eval = max(max_eval, val)
        return max_eval
    else:
        min_eval = float('inf')
        for move in moves:
            new_state = apply(state, move)
            val = minimax(new_state, depth-1, True)
            min_eval = min(min_eval, val)
        return min_eval


def best_move(state: "State", depth: int) -> tuple:
    global examined_nodes
    examined_nodes = 0

    maximizing = True if state.active_player == "MAX" else False
    best_val = -float('inf') if maximizing else float('inf')
    best_actions = []

    for move in legal_moves(state):
        new_state = apply(state, move)
        val = minimax(new_state, depth-1, not maximizing)

        if maximizing:
            if val > best_val:
                best_val = val
                best_actions = [move]
            elif val == best_val:
                best_actions.append(move)
        else:
            if val < best_val:
                best_val = val
                best_actions = [move]
            elif val == best_val:
                best_actions.append(move)

    chosen_move = random.choice(best_actions)
    print(f"Examined nodes: {examined_nodes}")
    return chosen_move


In [38]:
best_move = best_move(state, 2)
print(best_move)

Examined nodes: 17032
('MOVE', (2, 5))


### ALPHA-BETA PRUNING

In [45]:
def get_ordered_moves(state: State):
    moves = list(legal_moves(state))
    
    def move_priority(move):
        if hasattr(move, 'row'): 
            goal = 9 if state.active_player == "MAX" else 1
            dist_to_goal = abs(move.row - goal)
            return 100 - dist_to_goal
        return 0 
    
    return sorted(moves, key=move_priority, reverse=True)

def alphabeta(state: State, depth: int, alpha: float, beta: float, maximizing: bool) -> float:
    global examined_nodes, pruned_nodes
    examined_nodes += 1

    if depth == 0 or is_terminal(state):
        return evaluation(state)

    moves = get_ordered_moves(state)

    if maximizing:
        value = -float('inf')
        for move in moves:
            new_state = apply(state, move)
            value = max(value, alphabeta(new_state, depth-1, alpha, beta, False))
            alpha = max(alpha, value)
            if alpha >= beta:
                pruned_nodes += 1
                break
        return value
    else:
        value = float('inf')
        for move in moves:
            new_state = apply(state, move)
            value = min(value, alphabeta(new_state, depth-1, alpha, beta, True))
            beta = min(beta, value)
            if beta <= alpha:
                pruned_nodes += 1
                break
        return value

In [40]:
move = best_move_ab(state, depth=2)
print("Chosen move:", move)

Examined nodes: 392, Pruned nodes: 130
Chosen move: ('MOVE', (2, 5))


### Agent

In [46]:
class Agent:
    def __init__(self, depth: int = 4):
        self.depth = depth

    def get_action(self, state: State):
        global examined_nodes, pruned_nodes
        examined_nodes = 0
        pruned_nodes = 0

        maximizing = state.active_player == "MAX"
        alpha, beta = -float('inf'), float('inf')
        best_val = -float('inf') if maximizing else float('inf')
        best_actions = []
        
        moves = get_ordered_moves(state)

        for move in moves:
            new_state = apply(state, move)
            val = alphabeta(new_state, self.depth - 1, alpha, beta, not maximizing)

            if maximizing:
                if val > best_val:
                    best_val, best_actions = val, [move]
                elif val == best_val:
                    best_actions.append(move)
                alpha = max(alpha, best_val)
            else:
                if val < best_val:
                    best_val, best_actions = val, [move]
                elif val == best_val:
                    best_actions.append(move)
                beta = min(beta, best_val)

        if not best_actions: return None
        
        print(f"Nodes Examined: {examined_nodes} | Nodes Pruned: {pruned_nodes}")
        return random.choice(best_actions)

In [42]:
state = State.get_initial_state(random_turn=True)
state.display()

agent = Agent(depth=3)
move = agent.get_action(state)
print("Agent chooses:", move)

   1 2 3 4 5 6 7 8 9
1 .   .   .   .   1   .   .   .   .
                                   
2 .   .   .   .   .   .   .   .   .
                                   
3 .   .   .   .   .   .   .   .   .
                                   
4 .   .   .   .   .   .   .   .   .
                                   
5 .   .   .   .   .   .   .   .   .
                                   
6 .   .   .   .   .   .   .   .   .
                                   
7 .   .   .   .   .   .   .   .   .
                                   
8 .   .   .   .   .   .   .   .   .
                                   
9 .   .   .   .   2   .   .   .   .
Nodes Examined: 17426 | Nodes Pruned: 260
Agent chooses: ('MOVE', (8, 5))


In [ ]:
agent_max = Agent(depth=3)  # MAX player
agent_min = Agent(depth=3)  # MIN player

current_state = State.get_initial_state(random_turn=True)
current_state.display()

while not is_terminal(current_state):
    if current_state.active_player == "MAX":
        move = agent_max.get_action(current_state)
    else:
        move = agent_min.get_action(current_state)

    current_state = apply(current_state, move)
    current_state.display()

current_state.display()
if current_state.p1.row == 9:
    print("MAX wins!")
elif current_state.p2.row == 1:
    print("MIN wins!")
else:
    print('It is a draw')